# Collecting ASCAP Info + Storing in DB

**Description**: Pulling necessary song info from the ASCAP HTML files & storing in PostreSQL db

In [1]:
import json
import os
import pickle
import re

from bs4 import BeautifulSoup, element
from sqlalchemy import create_engine

import numpy as np
import pandas as pd

## Loading in HTML File

I'll need to figure out how to retrieve all of the corresponding song details from each HTML file that I scraped from ASCAP>

In [37]:
with open('../data/ascap_songs/CHIEF KEEF.html', 'rb+') as f:
    test_html = BeautifulSoup(f, 'lxml')

### Retrieving Song Cards

My current thought is that I should divvy these results into a list of `dict`s, which should allow me to easily toss into a dataframe or db afterwards.

In [38]:
test_cards = test_html.find_all('div', {'class':'card hyphenate card--expanded'})

64 songs indexed for Chief Keef

In [39]:
len(test_cards)

64

In [45]:
tc2 = test_cards[2]

In [57]:
tc2.find('div', {'class':'workID'}).text.strip().split()[-1]

'887415977'

##### The title and "Total Current ASCAP Share"

In [13]:
test_cards[0].find('div', {'class':'card__header__title'}).text.strip().split(sep='\n')

['ADAM & EVE',
 '                ',
 '                    ',
 '                    Total Current ASCAP Share:',
 '                    25%']

##### Just the song title

In [14]:
test_cards[0].find('div', {'class':'card__header__title'}).text.strip().split(sep='\n')[0]

'ADAM & EVE'

##### Writers

In [15]:
writers_row = test_cards[0].find_all('tr', {'class':'writers__row'})

In [16]:
writer_list = []

for writer in writers_row:
    if isinstance(writer.find('a'), element.Tag):
        writer_list.append((writer.find('a'), writer.find_all('td', {'class':'sup-col'})))
    elif isinstance(writer.find('span'), element.Tag):
        writer_list.append((writer.find('span'), writer.find_all('td', {'class':'sup-col'})))
    else:
        writer_list.append('cannot find writer')
        
writer_info = []

for writer, writer_ids in writer_list:
    b = writer.attrs
    b['pro'] = writer_ids[0].text.strip()
    b['ipi'] = writer_ids[1].text.strip()
    writer_info.append(b)

In [17]:
writer_list[0][0].attrs.pop('href')
writer_list[0][0].attrs.pop('class')
writer_list[0][0].attrs

{'data-writer': 'MCKINNEY REGINALD CHRISTOPHER',
 'data-id': '652264355',
 'pro': 'ASCAP',
 'ipi': '652264355'}

In [18]:
writer_info

[{'data-writer': 'MCKINNEY REGINALD CHRISTOPHER',
  'data-id': '652264355',
  'pro': 'ASCAP',
  'ipi': '652264355'},
 {'class': ['no-click'],
  'data-writer': 'WRITER UNKNOWN  ',
  'data-id': '0',
  'pro': 'NS',
  'ipi': '—'}]

Performers

In [19]:
performers = test_cards[0].find_all('div', {'class':'creditors__list'})

In [20]:
performers[0].text.strip('\n')

'CHIEF KEEF'

Retreive one or multiple performers per song

In [21]:
if performers[0].text.count('\n') > 2:
    performers[0].text.strip().split(sep='\n')
else:
    performers[0].text.strip('\n')

HTML Title

### Defining Functions to Sort Through HTML Files

Here's how I'll take the song information that I need and output it into a list of `dict`s

In [60]:
def bs_file(file):
    '''
    Opens file and returns beautiful soup object
    '''
    with open('../data/ascap_songs/{}'.format(file), 'rb') as f:
        bs = BeautifulSoup(f, 'lxml')
    return bs

def song_sorter(bs):
    '''
    Iterates through songs in bs object and returns cards of all songs
    '''
    results = []
    if len(bs.find_all('div', {'class':'card hyphenate card--expanded'})) > 0:
        results.extend(bs.find_all('div', {'class':'card hyphenate card--expanded'}))
    if len(bs.find_all('div', {'class':'card card--expanded hyphenate'})) > 0:
        results.extend(bs.find_all('div', {'class':'card card--expanded hyphenate'}))
    return results
                                   
def get_song_credits(card):
    '''
    Retrieves song credits from an individual song card
    '''   
    title = card.find('div', {'class':'card__header__title'}).text.strip().split(sep='\n')[0]
    aid = card.find('div', {'class':'workID'}).text.strip().split()[-1]
    writers_row = card.find_all('tr', {'class':'writers__row'})
    writer_list = []
    
    # Creating writer list for further filtering
    for writer in writers_row:
        if isinstance(writer.find('a'), element.Tag):
            writer_list.append((writer.find('a'), writer.find_all('td', {'class':'sup-col'})))
        elif isinstance(writer.find('span'), element.Tag):
            writer_list.append((writer.find('span'), writer.find_all('td', {'class':'sup-col'})))
        else:
            writer_list.append('cannot find writer')
    writers = []
    for writer, writer_ids in writer_list:
        b = writer.attrs
        b['pro'] = writer_ids[0].text.strip()
        b['ipi'] = writer_ids[1].text.strip()
        writers.append(b)
    
    # Grabbing performers
    performers = card.find_all('div', {'class':'creditors__list'})
    if performers[0].text.count('\n') > 2:
        artists = performers[0].text.strip().split(sep='\n')
    else:
        artists = performers[0].text.strip('\n')
        
    # Grabbing alternate titles
    try:
        if performers[1].text.count('\n'):
            alt_titles = performers[1].text.strip().split(sep='\n')
        else:
            alt_titles = performers[1].text.strip('\n')
    except IndexError:
        alt_titles = None
        
    return title, aid, writers, artists, alt_titles

In [69]:
ck = song_sorter(test_html)
t, aid, w, a, at = get_song_credits(ck[2])

In [70]:
aid

'887415977'

In [46]:
len(ck)

64

In [71]:
def get_html_songs(html_list):
    '''
    Retreives songs in html files and stores in a list of dicts
    '''
    song_list = []
    count = 0
    for file in html_list:
        bs = bs_file(file)
        songs = song_sorter(bs)
        for song in songs:
            count += 1
            t, aid, w, p, at = get_song_credits(song)
            song_dict = {}
            song_dict['Title'] = t
            song_dict['AID'] = aid
            song_dict['Writers'] = w
            song_dict['Performers'] = p
            song_dict['Alt-Titles'] = at
            song_list.append(song_dict)
            if len(song_list) % 5000 == 0:
                print('{} songs completed'.format(count))
    return song_list

## Retrieving HTML File List

In [5]:
html_list = os.listdir('../data/ascap_songs/')

In [6]:
len(html_list)

13019

In [7]:
html_list.index('.ipynb_checkpoints')

9917

In [8]:
html_list.pop(9917)

'.ipynb_checkpoints'

#### Ensuring there are no more non-html files in list

In [14]:
files = [html.find('.html') for html in html_list]
files.sort()
files[:10]

[0, 1, 2, 2, 2, 2, 2, 2, 2, 2]

### Quick Function Test

In [13]:
test = get_html_songs(html_list[0:5])

In [63]:
html_list[0:5]

['BONE THUGS N HARMONY FEAT. EAZY-E.html',
 'VESTAL AND LEE GREENWOOD.html',
 ' PETE SEEGER.html',
 'DARIUS RUCKER FEAT JILL SCOTT.html',
 'GARTH BROOKS.html']

In [14]:
len(test)

7

## Retrieving Song Results

In [72]:
song_list = get_html_songs(html_list)

5000 songs completed
10000 songs completed
15000 songs completed
20000 songs completed
25000 songs completed
30000 songs completed
35000 songs completed
40000 songs completed
45000 songs completed
50000 songs completed
55000 songs completed
60000 songs completed
65000 songs completed
70000 songs completed
75000 songs completed
80000 songs completed
85000 songs completed
90000 songs completed
95000 songs completed
100000 songs completed
105000 songs completed
110000 songs completed
115000 songs completed
120000 songs completed
125000 songs completed
130000 songs completed
135000 songs completed
140000 songs completed
145000 songs completed
150000 songs completed
155000 songs completed
160000 songs completed
165000 songs completed
170000 songs completed
175000 songs completed
180000 songs completed
185000 songs completed
190000 songs completed
195000 songs completed
200000 songs completed
205000 songs completed
210000 songs completed
215000 songs completed
220000 songs completed
225000 s

#### Saving `song_list` as pickle

In [73]:
with open('../pickle/ascap_credits.pkl', 'wb') as f:
    pickle.dump(song_list, f)

In [2]:
with open('../pickle/ascap_credits.pkl', 'rb') as f:
    song_list = pickle.load(f)

### Examining `song_list`

In [3]:
song_list[129020]

{'Title': "BOSSIN' UP",
 'AID': '886044026',
 'Writers': [{'class': ['no-click'],
   'data-writer': 'COLLINS BRIAN ',
   'data-id': '0',
   'pro': 'NS',
   'ipi': '—'},
  {'class': ['no-click'],
   'data-writer': 'FERGUSON DAROLD ',
   'data-id': '0',
   'pro': 'NS',
   'ipi': '—'},
  {'class': ['no-click'],
   'data-writer': 'KHARBOUCH KARIM ',
   'data-id': '0',
   'pro': 'NS',
   'ipi': '—'},
  {'href': 'javascript:void(0)',
   'class': ['name'],
   'data-writer': 'TAFT STEPAN ',
   'data-id': '539311163',
   'pro': 'ASCAP',
   'ipi': '539311163'},
  {'class': ['no-click'],
   'data-writer': 'THOMAS JAMES ',
   'data-id': '0',
   'pro': 'NS',
   'ipi': '—'}],
 'Performers': 'KID INK',
 'Alt-Titles': None}

In [4]:
len(song_list)

309581

#### Loading List into DataFrame

In [3]:
df = pd.DataFrame(data=song_list)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309581 entries, 0 to 309580
Data columns (total 5 columns):
AID           309581 non-null object
Alt-Titles    94080 non-null object
Performers    309581 non-null object
Title         309581 non-null object
Writers       309581 non-null object
dtypes: object(5)
memory usage: 11.8+ MB


In [5]:
df.head()

,AID,Alt-Titles,Performers,Title,Writers
0,360318916,"[FOE THA LOVE OF $, FOE THA LOVE OF $, FOE THA...","[BONE, BONE THUGS N HARMONY, BONE THUGS N HARM...",FOR THA LOVE OF MONEY,"[{'class': ['no-click'], 'data-writer': 'CARRA..."
1,530659306,None,"[GOODMAN V, GREENWOOD AND VESTAL, GREENWOOD L,...",WE THE PEOPLE,"[{'href': 'javascript:void(0)', 'class': ['nam..."
2,334030418,[CELERY TIME],ARLO GUTHRIE / PETE SEEGER,CELERY-TIME,"[{'href': 'javascript:void(0)', 'class': ['nam..."
3,442081954,None,ARLO GUTHRIE / PETE SEEGER,NEUTRON BOMB,"[{'href': 'javascript:void(0)', 'class': ['nam..."
4,230055482,None,"[ARLO GUTHRIE / PETE SEEGER, GUTHRIE A, GUTHRI...",WILL THE CIRCLE BE UNBROKEN,"[{'href': 'javascript:void(0)', 'class': ['nam..."


#### Deleting potential duplicates

Duplicate 'AID' records should be the only song duplicates

In [6]:
df.drop_duplicates('AID', inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264175 entries, 0 to 309575
Data columns (total 5 columns):
AID           264175 non-null object
Alt-Titles    65799 non-null object
Performers    264175 non-null object
Title         264175 non-null object
Writers       264175 non-null object
dtypes: object(5)
memory usage: 12.1+ MB


In [8]:
df.head()

,AID,Alt-Titles,Performers,Title,Writers
0,360318916,"[FOE THA LOVE OF $, FOE THA LOVE OF $, FOE THA...","[BONE, BONE THUGS N HARMONY, BONE THUGS N HARM...",FOR THA LOVE OF MONEY,"[{'class': ['no-click'], 'data-writer': 'CARRA..."
1,530659306,None,"[GOODMAN V, GREENWOOD AND VESTAL, GREENWOOD L,...",WE THE PEOPLE,"[{'href': 'javascript:void(0)', 'class': ['nam..."
2,334030418,[CELERY TIME],ARLO GUTHRIE / PETE SEEGER,CELERY-TIME,"[{'href': 'javascript:void(0)', 'class': ['nam..."
3,442081954,None,ARLO GUTHRIE / PETE SEEGER,NEUTRON BOMB,"[{'href': 'javascript:void(0)', 'class': ['nam..."
4,230055482,None,"[ARLO GUTHRIE / PETE SEEGER, GUTHRIE A, GUTHRI...",WILL THE CIRCLE BE UNBROKEN,"[{'href': 'javascript:void(0)', 'class': ['nam..."


Going forward, this will be the primary table for song information. The composition ID will be the current index

### Cleaning Records

Before tossing tossing this dataframe into multiple tables stored in my PostgreSQL db, I'll need to clean each of the associated fields and seperate them into different tables.

In [9]:
df['ATTypes'] = df['Alt-Titles'].apply(lambda x: type(x))
df['PTypes'] = df['Performers'].apply(lambda x: type(x))
df['TTypes'] = df['Title'].apply(lambda x: type(x))
df['WTypes'] = df['Writers'].apply(lambda x: type(x))

In [10]:
df.head()

,AID,Alt-Titles,Performers,Title,Writers,ATTypes,PTypes,TTypes,WTypes
0,360318916,"[FOE THA LOVE OF $, FOE THA LOVE OF $, FOE THA...","[BONE, BONE THUGS N HARMONY, BONE THUGS N HARM...",FOR THA LOVE OF MONEY,"[{'class': ['no-click'], 'data-writer': 'CARRA...",<class 'list'>,<class 'list'>,<class 'str'>,<class 'list'>
1,530659306,None,"[GOODMAN V, GREENWOOD AND VESTAL, GREENWOOD L,...",WE THE PEOPLE,"[{'href': 'javascript:void(0)', 'class': ['nam...",<class 'NoneType'>,<class 'list'>,<class 'str'>,<class 'list'>
2,334030418,[CELERY TIME],ARLO GUTHRIE / PETE SEEGER,CELERY-TIME,"[{'href': 'javascript:void(0)', 'class': ['nam...",<class 'list'>,<class 'str'>,<class 'str'>,<class 'list'>
3,442081954,None,ARLO GUTHRIE / PETE SEEGER,NEUTRON BOMB,"[{'href': 'javascript:void(0)', 'class': ['nam...",<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
4,230055482,None,"[ARLO GUTHRIE / PETE SEEGER, GUTHRIE A, GUTHRI...",WILL THE CIRCLE BE UNBROKEN,"[{'href': 'javascript:void(0)', 'class': ['nam...",<class 'NoneType'>,<class 'list'>,<class 'str'>,<class 'list'>


In [11]:
df['ATTypes'].value_counts()

<class 'NoneType'>    198376
<class 'list'>         65799
Name: ATTypes, dtype: int64

In [12]:
df['PTypes'].value_counts()

<class 'str'>     181391
<class 'list'>     82784
Name: PTypes, dtype: int64

In [13]:
df['TTypes'].value_counts()

<class 'str'>    264175
Name: TTypes, dtype: int64

In [14]:
df['WTypes'].value_counts()

<class 'list'>    264175
Name: WTypes, dtype: int64

#### Establishing `CID`, which will be the composition level ID going forward

In [15]:
df['CID'] = df.index

#### Creating `main`, which will be the primary table for accessing composition information

In [16]:
main_df = df[['CID','AID', 'Title']]

In [17]:
main_df.head()

,CID,AID,Title
0,0,360318916,FOR THA LOVE OF MONEY
1,1,530659306,WE THE PEOPLE
2,2,334030418,CELERY-TIME
3,3,442081954,NEUTRON BOMB
4,4,230055482,WILL THE CIRCLE BE UNBROKEN


In [89]:
main_df.to_csv('../data/compositions.csv')

#### Cleaning `Alt-Titles` and creating Alt-Titles table

In [18]:
alt_title_df = df[['Alt-Titles', 'CID']]

In [19]:
alt_title_df.head()

,Alt-Titles,CID
0,"[FOE THA LOVE OF $, FOE THA LOVE OF $, FOE THA...",0
1,None,1
2,[CELERY TIME],2
3,None,3
4,None,4


##### Devising scheme to create unique record for each alt-title

In [20]:
for rec, (titles, ids) in alt_title_df.head().iterrows():
    print(type(titles))

<class 'list'>
<class 'NoneType'>
<class 'list'>
<class 'NoneType'>
<class 'NoneType'>


In [21]:
at_dict = {}
count = 0
for rec, (titles, ids) in alt_title_df.iterrows():
    if isinstance(titles, list):
        for title in titles:
            count += 1
            at_dict[count] = title, ids

##### Total Size of alt-titles dict

In [22]:
len(at_dict)

159725

In [23]:
at_df = pd.DataFrame.from_dict(at_dict, orient='index', columns=['alt-title', 'CID'])

In [24]:
at_df.head()

,alt-title,CID
1,FOE THA LOVE OF $,0
2,FOE THA LOVE OF $,0
3,FOE THA LOVE OF $ (FEAT EASY-E),0
4,FOE THA LOVE OF $ (FEAT. EAZY-E),0
5,FOE THA LOVE OF $ [EXPLICIT],0


##### Saving as a csv

In [25]:
at_df.to_csv('../data/comp_alt_titles.csv')

#### Creating separate `Performers` table

I'll need to definitely create a lookup table for these entries. As it currently stands, I'm not cleaning any of the entries. I don't think it's worth it since I'll be relying on the spotify artist names for application use purposes. It might be helpful to lemmatize the artist names, or do something to garner a root artist name for each. This might help with matching songs to recordings.

In [26]:
df.head()

,AID,Alt-Titles,Performers,Title,Writers,ATTypes,PTypes,TTypes,WTypes,CID
0,360318916,"[FOE THA LOVE OF $, FOE THA LOVE OF $, FOE THA...","[BONE, BONE THUGS N HARMONY, BONE THUGS N HARM...",FOR THA LOVE OF MONEY,"[{'class': ['no-click'], 'data-writer': 'CARRA...",<class 'list'>,<class 'list'>,<class 'str'>,<class 'list'>,0
1,530659306,None,"[GOODMAN V, GREENWOOD AND VESTAL, GREENWOOD L,...",WE THE PEOPLE,"[{'href': 'javascript:void(0)', 'class': ['nam...",<class 'NoneType'>,<class 'list'>,<class 'str'>,<class 'list'>,1
2,334030418,[CELERY TIME],ARLO GUTHRIE / PETE SEEGER,CELERY-TIME,"[{'href': 'javascript:void(0)', 'class': ['nam...",<class 'list'>,<class 'str'>,<class 'str'>,<class 'list'>,2
3,442081954,None,ARLO GUTHRIE / PETE SEEGER,NEUTRON BOMB,"[{'href': 'javascript:void(0)', 'class': ['nam...",<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>,3
4,230055482,None,"[ARLO GUTHRIE / PETE SEEGER, GUTHRIE A, GUTHRI...",WILL THE CIRCLE BE UNBROKEN,"[{'href': 'javascript:void(0)', 'class': ['nam...",<class 'NoneType'>,<class 'list'>,<class 'str'>,<class 'list'>,4


In [27]:
for rec, (performers, ids) in df[['Performers', 'CID']].head().iterrows():
    print(performers)

['BONE', 'BONE THUGS N HARMONY', 'BONE THUGS N HARMONY FEAT. EAZY-E', 'BONE THUGS-N-HARMONY', 'BONE THUGS-N-HARMONY (EDITED)', 'BONE THUGS-N-HARMONY EAZY-E', 'DR. DRE JILL SCOTT ANDERSON PAAK & JON CONNOR', 'EASY E', 'EAZY-E', 'EAZY-E & BONE THUGS-N-HARMONY']
['GOODMAN V', 'GREENWOOD AND VESTAL', 'GREENWOOD L', 'GREENWOOD LAND GREENWOOD', 'LEE GREENWOOD', 'LEE GREENWOOD AND VESTAL', 'VESTAL AND GREENWOOD', 'VESTAL AND LEE GREENWOOD', 'VESTAL GOODMAN']
ARLO GUTHRIE / PETE SEEGER
ARLO GUTHRIE / PETE SEEGER
['ARLO GUTHRIE / PETE SEEGER', 'GUTHRIE A', 'GUTHRIE ARLO', 'SEEGER P', 'SEEGER PETE']


In [28]:
p_dict = {}
count = 0
for rec, (performers, ids) in df[['Performers', 'CID']].iterrows():
    if isinstance(performers, list):
        for performer in performers:
            count += 1
            p_dict[count] = performer, ids
    else:
        count += 1
        p_dict[count] = performers, ids

In [29]:
p_dict[4]

('BONE THUGS-N-HARMONY', 0)

In [30]:
p_df = pd.DataFrame.from_dict(p_dict, orient='index', columns=['Artist Name', 'CID'])

In [31]:
p_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 570810 entries, 1 to 570810
Data columns (total 2 columns):
Artist Name    570810 non-null object
CID            570810 non-null int64
dtypes: int64(1), object(1)
memory usage: 13.1+ MB


In [32]:
p_df.head()

,Artist Name,CID
1,BONE,0
2,BONE THUGS N HARMONY,0
3,BONE THUGS N HARMONY FEAT. EAZY-E,0
4,BONE THUGS-N-HARMONY,0
5,BONE THUGS-N-HARMONY (EDITED),0


##### Duplicate Artist Name Check

In [33]:
p_df[p_df.duplicated('Artist Name')].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438196 entries, 21 to 570810
Data columns (total 2 columns):
Artist Name    438196 non-null object
CID            438196 non-null int64
dtypes: int64(1), object(1)
memory usage: 10.0+ MB


In [34]:
len(p_df['Artist Name'].unique())

132614

#### Creating Unique Artists Table

In [44]:
comp_artists_df = pd.DataFrame(p_df['Artist Name'].unique(), columns=['Performer Name'])
comp_artists_df['PID'] = comp_artists_df.index

In [46]:
comp_artists_df.head()

,Performer Name,PID
0,BONE,0
1,BONE THUGS N HARMONY,1
2,BONE THUGS N HARMONY FEAT. EAZY-E,2
3,BONE THUGS-N-HARMONY,3
4,BONE THUGS-N-HARMONY (EDITED),4


In [47]:
comp_artists_df.to_csv("../data/comp_artists.csv")

#### Creating Performer / Composition Lookup Table

In [82]:
len(p_df)

570810

In [78]:
p_df.rename({'Artist Name':'Performer Name'}, axis=1, inplace=True)

In [79]:
p_lookup = pd.merge(p_df, comp_artists_df, on='Performer Name')

In [86]:
p_lookup.drop('Performer Name', 1, inplace=True)

In [88]:
p_lookup.to_csv('../data/artist_comp_lookup.csv')

#### Creating Writer Table

In [91]:
writers = df[['CID', 'Writers']]

In [100]:
writers['Writers'].iloc[0]

dict

In [101]:
writers_dict = {}
count = 0
for writer_list in writers['Writers'].values:
    for entry in writer_list:
        count += 1
        writers_dict[count] = entry['data-writer'], entry['ipi'], entry['pro']

In [104]:
len(writers_dict)

564706

In [108]:
writers_df = pd.DataFrame.from_dict(writers_dict, orient='index', columns=['Writer Name', 'IPI', 'PRO'])
writers_df.head()

,Writer Name,IPI,PRO
1,CARRABY ANTOINE,205786169,BMI
2,GREEN MARK ERIC,333982644,ASCAP
3,HENDERSON ANTHONY,337563351,ASCAP
4,HOWSE STANLEY,337563939,ASCAP
5,HOWSE STEVEN,337564152,ASCAP


In [112]:
writers_df.drop_duplicates(subset='IPI', inplace=True)

In [113]:
writers_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67468 entries, 1 to 564677
Data columns (total 3 columns):
Writer Name    67468 non-null object
IPI            67468 non-null object
PRO            67468 non-null object
dtypes: object(3)
memory usage: 2.1+ MB


In [114]:
writers_df.to_csv('../data/writers.csv')

#### Creating Writer Lookup Table

In [115]:
writers.head()

,CID,Writers
0,0,"[{'class': ['no-click'], 'data-writer': 'CARRA..."
1,1,"[{'href': 'javascript:void(0)', 'class': ['nam..."
2,2,"[{'href': 'javascript:void(0)', 'class': ['nam..."
3,3,"[{'href': 'javascript:void(0)', 'class': ['nam..."
4,4,"[{'href': 'javascript:void(0)', 'class': ['nam..."


In [120]:
cw_dict = {}
count = 0
for cid, writer_list in writers.values:
    for entry in writer_list:
        count += 1
        cw_dict[count] = cid, entry['ipi']

In [121]:
len(cw_dict)

564706

In [122]:
cw_lookup = pd.DataFrame.from_dict(cw_dict, orient='index', columns=['CID', 'IPI'])

In [123]:
cw_lookup.head()

,CID,IPI
1,0,205786169
2,0,333982644
3,0,337563351
4,0,337563939
5,0,337564152


In [124]:
cw_lookup.to_csv('../data/cw_lookup.csv')